In [4]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config

In [5]:
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

In [4]:
#establish connection with bucket
bucket_name =  "d2b-internal-assessment-bucket"
response = s3.list_objects(Bucket=bucket_name, Prefix="orders_data")


In [13]:
#to download the orders.csv
s3.download_file(bucket_name, "orders_data/orders.csv", "orders.csv")

In [27]:
#check data
s3.download_file(bucket_name, "orders_data/shipments_deliveries.csv", "shipments_deliveries.csv")

In [19]:
#check data
s3.download_file(bucket_name, "orders_data/reviews.csv", "reviews.csv")

In [23]:
#check data
s3.download_file(bucket_name, "orders_data/shipment_deliveries.csv", "shipments_deliveries.csv")

In [24]:
##check for list of files in aws bucket
file_names = [file['Key'] for file in response['Contents']]

SyntaxError: invalid syntax (2112612705.py, line 1)

In [22]:
##get list of files
for file_name in file_names:
    print(file_name)

orders_data/
orders_data/analytics_export/murtodun9658/agg_public_holiday.csv
orders_data/analytics_export/murtodun9658/agg_shipments.csv
orders_data/analytics_export/murtodun9658/best_performing_product.csv
orders_data/analytics_export/murtodun9658/late_shipments.csv
orders_data/analytics_export/murtodun9658/undelivered_shipments.csv
orders_data/orders.csv
orders_data/reviews.csv
orders_data/shipment_deliveries.csv


In [6]:
#import necessary libraries
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
import csv
from datetime import datetime

In [275]:
curs = conn.cursor()
curs.execute("ROLLBACK")
conn.commit()

In [7]:
##connect to postgres database
conn = psycopg2.connect(
    host='d2b-internal-assessment-dwh.cxeuj0ektqdz.eu-central-1.rds.amazonaws.com',
    port='5432',
    dbname='d2b_assessment',
    user='niniroti9849',
    password='Wlaypw0N2l'
)

In [194]:
#specify csv_file_path = '/Users/niniolarotimi/reviews.csv'
csv_file_path2 = '/Users/niniolarotimi/shipments_deliveries.csv'
csv_file_path3 = '/Users/niniolarotimi/orders.csv'

In [195]:
#table names to load data to
table_name = 'niniroti9849_staging.reviews'
table_name2 = 'niniroti9849_staging.shipments_deliveries'
table_name3 = 'niniroti9849_staging.orders'

In [176]:
#establish connection
conn = psycopg2.connect(
   host = 'd2b-internal-assessment-dwh.cxeuj0ektqdz.eu-central-1.rds.amazonaws.com',
   port = '5432',
   dbname = 'd2b_assessment',
   user = 'niniroti9849',
   password = 'Wlaypw0N2l'

)

cur = conn.cursor()

In [ ]:
##load downloaded file to reviews table
with open(csv_file_path, 'r') as file:
    # Create a CSV reader object
    csv_reader = csv.reader(file)
    
    # Skip the header row
    next(csv_reader)
    
    # Iterate over each row in the CSV file
    for row in csv_reader:
        # Extract the values from the row
        review = row[0]  # Assuming the review column is the first column in the CSV file
        product_id = row[1]  # Assuming the other column is the second column in the CSV file
        
        # Construct the SQL INSERT statement
        insert_query = f"INSERT INTO {table_name} (review, product_id) VALUES (%s, %s)"
        
        # Execute the INSERT statement with the current row data
        cur.execute(insert_query, (review, product_id))
        


In [ ]:
# Commit the changes to the database
conn.commit()

In [197]:
##load downloaded file to shipping table in the db
with open(csv_file_path2, 'r') as file2:
    # Create a CSV reader object
    csv_reader2 = csv.reader(file2)
    
    # Skip the header row
    next(csv_reader2)
    
    # Iterate over each row in the CSV file
    for row in csv_reader2:
        # Extract the values from the row
        shipment_id = row[0]
        order_id = row[1]
        shipment_date = row[2]
        delivery_date = row[3]
        
        # Convert the date string to a date object
        shipment_date = datetime.strptime(shipment_date, '%Y-%m-%d').date() if shipment_date else None
        
        # Check if the delivery_date is not empty
        delivery_date = datetime.strptime(delivery_date, '%Y-%m-%d').date() if delivery_date else None
        
        # Construct the SQL INSERT statement
        insert_query = f"INSERT INTO {table_name2} (shipment_id, order_id, shipment_date, delivery_date) VALUES (%s, %s, %s, %s)"
        
        # Execute the INSERT statement with the current row data
        cur.execute(insert_query, (shipment_id, order_id, shipment_date, delivery_date))
        
# Commit the changes to the database
conn.commit()


In [202]:
##load orders table to the db
with open(csv_file_path3, 'r') as file3:
    # Create a CSV reader object
    csv_reader3 = csv.reader(file3)
    
    # Skip the header row
    next(csv_reader3)
    
    # Iterate over each row in the CSV file
    for row in csv_reader3:
        # Extract the values from the row
        order_id = row[0]
        customer_id = row[1]
        order_date = row[2]
        product_id = row[3]
        unit_price = row[4]
        quantity = row[5]
        total_price = row[6]
        
        # Convert the date string to a date object
        order_date = datetime.strptime(order_date, '%Y-%m-%d').date() if order_date else None
        
        # Construct the SQL INSERT statement
        insert_query = f"INSERT INTO {table_name3} (order_id, customer_id, order_date, product_id, unit_price, quantity, total_price) VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        # Execute the INSERT statement with the current row data
        cur.execute(insert_query, (order_id, customer_id, order_date, product_id, unit_price, quantity, total_price))
        
# Commit the changes to the database
conn.commit()

In [178]:
conn = psycopg2.connect(
   host = 'd2b-internal-assessment-dwh.cxeuj0ektqdz.eu-central-1.rds.amazonaws.com',
   port = '5432',
   dbname = 'd2b_assessment',
   user = 'niniroti9849',
   password = 'Wlaypw0N2l'

)

cur = conn.cursor()

In [179]:
#no of orders on public holidays in the past year

cur.execute('''
    SELECT COUNT(a.order_id) AS order_count, TO_CHAR(b.calendar_dt, 'Month') AS month
    FROM niniroti9849_staging.orders a
    LEFT JOIN if_common.dim_dates b ON a.order_date = b.calendar_dt
    WHERE (b.day_of_the_week_num IN (1, 2, 3, 4, 5) AND b.working_day = 'false')
    AND DATE_PART('year', b.calendar_dt) = '2021'
    GROUP BY TO_CHAR(b.calendar_dt, 'Month')
''')


rows = cur.fetchall()
# Iterate over the rows and print the data
for row in rows:
    print(row)

(20, 'April    ')
(18, 'August   ')
(20, 'January  ')
(16, 'March    ')
(12, 'November ')
(16, 'October  ')


In [189]:
curs = conn.cursor()
curs.execute("ROLLBACK")
conn.commit()

In [229]:
#total number of late shipments

cur.execute('''
select count(a.order_id)	
from niniroti9849_staging.orders a
left join niniroti9849_staging.shipments_deliveries b on a.order_id = b.order_id
where DATE_PART('day', AGE(b.shipment_date, a.order_date)) >= 6
and delivery_date is null
''')

rows = cur.fetchall()
# Iterate over the rows and print the data
for row in rows:
    print(row)

(175,)


In [234]:
#total number of undelivered shipments

cur.execute('''
select count(a.order_id)	
from niniroti9849_staging.orders a
left join niniroti9849_staging.shipments_deliveries b on a.order_id = b.order_id
--where DATE_PART('day', AGE('2022-09-05', a.order_date)) >= 15
where delivery_date is null
and b.shipment_date is null
AND '2022-09-05' >= (order_date + INTERVAL '15 days');
''')

rows = cur.fetchall()
# Iterate over the rows and print the data
for row in rows:
    print(row)

(6586,)


In [199]:
#define connection
conn_write = psycopg2.connect(
   host = 'd2b-internal-assessment-dwh.cxeuj0ektqdz.eu-central-1.rds.amazonaws.com',
   port = '5432',
   dbname = 'd2b_assessment',
   user = 'niniroti9849',
   password = 'Wlaypw0N2l'

)

In [200]:
sql_query1 =  """
select count(a.order_id) as orders
from niniroti9849_staging.orders a
left join niniroti9849_staging.shipments_deliveries b on a.order_id = b.order_id
where DATE_PART('day', AGE(b.shipment_date, a.order_date)) >= 6
and delivery_date is null
 """



In [201]:
sql_query2 = """
SELECT COUNT(a.order_id) AS order_count
FROM niniroti9849_staging.orders a
LEFT JOIN niniroti9849_staging.shipments_deliveries b ON a.order_id = b.order_id
WHERE delivery_date IS NULL
AND b.shipment_date IS NULL
AND '2022-09-05' >= (a.order_date + INTERVAL '15 days')
"""


In [202]:
insert_query = f"""
    INSERT INTO niniroti9849_analytics.agg_shipments (ingestion_date, tt_late_shipments, tt_undelivered_items)
    SELECT 
         CURRENT_DATE, 
        (SELECT orders FROM ({sql_query1}) AS subquery1),
        (SELECT order_count FROM ({sql_query2}) AS subquery2)
        
"""

In [204]:
##write all columns to agg shipments table


cur.execute(insert_query)

# Commit the changes to the database
conn.commit()

In [90]:
conn = psycopg2.connect(
   host = 'd2b-internal-assessment-dwh.cxeuj0ektqdz.eu-central-1.rds.amazonaws.com',
   port = '5432',
   dbname = 'd2b_assessment',
   user = 'niniroti9849',
   password = 'Wlaypw0N2l'

)

cur = conn.cursor()

In [91]:
#insert best performing product query into niniroti9849_analytics.best_performing_product_new; kindly note that existing table created had a bit of issue so this data was loaded to table 'niniroti9849_analytics.best_performing_product_new'

insert_query = f"""
INSERT INTO niniroti9849_analytics.best_performing_product_new(
  ingestion_date, 
  product_name, 
  most_ordered_day,
  is_public_holiday,
  tt_review_points,
  pct_one_star_review, 
  pct_two_star_review, 
  pct_three_star_review, 
  pct_four_star_review, 
  pct_five_star_review, 
  pct_early_shipments, 
  pct_late_shipments
)
SELECT
    current_date AS ingestion_date,
    p.product_name, 
    a.order_date AS most_ordered_day,
	d.is_public_holiday,
	b.tt_review_points,
    g.pct_one_star_review, 
    g.pct_two_star_review, 
    g.pct_three_star_review, 
    g.pct_four_star_review,
    g.pct_five_star_review,
    n.pct_early_shipments,
    n.pct_late_shipments
FROM niniroti9849_staging.orders a
LEFT JOIN (
    SELECT
        z.product_id,
        TRUNC((SUM(CASE WHEN z.shipment_status = 'Early Shipment' THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 1) AS pct_early_shipments,
        TRUNC((SUM(CASE WHEN z.shipment_status = 'Late Shipment' THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 1) AS pct_late_shipments 
    FROM (
        SELECT
            i.product_id,
            CASE 
                WHEN DATE_PART('day', AGE(x.shipment_date, i.order_date)) >= 6 AND x.delivery_date IS NULL THEN 'Late Shipment'
                WHEN DATE_PART('day', AGE(x.shipment_date, i.order_date)) < 6 AND x.delivery_date IS NULL THEN 'Early Shipment'
                ELSE 'On Time'
            END AS shipment_status,
            i.order_date
        FROM
            niniroti9849_staging.shipments_deliveries x
        LEFT JOIN
            niniroti9849_staging.orders i ON x.order_id = i.order_id
    ) z
    GROUP BY z.product_id
) n ON CAST(n.product_id AS INTEGER) = CAST(a.product_id AS INTEGER)
LEFT JOIN (
    SELECT
        product_id,
        TRUNC((SUM(CASE WHEN review = 1 THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 1) AS pct_one_star_review,
        TRUNC((SUM(CASE WHEN review = 2 THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 1) AS pct_two_star_review,
        TRUNC((SUM(CASE WHEN review = 3 THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 1) AS pct_three_star_review,
        TRUNC((SUM(CASE WHEN review = 4 THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 1) AS pct_four_star_review,
        TRUNC((SUM(CASE WHEN review = 5 THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 1) AS pct_five_star_review
    FROM
    niniroti9849_staging.reviews
    GROUP BY product_id
) g ON CAST(g.product_id AS INTEGER) = CAST(a.product_id AS INTEGER)
LEFT JOIN if_common.dim_products p ON CAST(p.product_id AS INTEGER) = CAST(a.product_id AS INTEGER)
Left join(select calendar_dt,case when day_of_the_week_num IN (1, 2, 3, 4, 5) AND working_day = 'false' 
		then TRUE ELSE FALSE END AS is_public_holiday
        from if_common.dim_dates)d on d.calendar_dt = a.order_date
LEFT JOIN (
    SELECT
        product_id,
        COUNT(review) AS no_of_reviews,
	    CAST(SUM(review) AS INTEGER) AS tt_review_points
	    
    FROM
        niniroti9849_staging.reviews
    GROUP BY product_id
    ORDER BY no_of_reviews DESC
    LIMIT 1
) b ON CAST(a.product_id AS INTEGER) = CAST(b.product_id AS INTEGER)
WHERE
    a.product_id = '23' 
    AND a.quantity = '10';
"""

In [92]:
#execute query
cur.execute(insert_query)

In [93]:
# Commit the changes to the database
conn.commit()

In [314]:
#create connection to database
conn = psycopg2.connect(
   host = 'd2b-internal-assessment-dwh.cxeuj0ektqdz.eu-central-1.rds.amazonaws.com',
   port = '5432',
   dbname = 'd2b_assessment',
   user = 'niniroti9849',
   password = 'Wlaypw0N2l'

)

cur = conn.cursor()

In [94]:
sql_query = """
SELECT
    current_date AS ingestion_date,
    p.product_name, 
    a.order_date AS most_ordered_day,
	d.is_public_holiday,
	b.total_review_points,
    g.pct_one_star_review, 
    g.pct_two_star_review, 
    g.pct_three_star_review, 
    g.pct_four_star_review,
    g.pct_five_star_review,
    n.pct_early_shipments,
    n.pct_late_shipments
FROM niniroti9849_staging.orders a
LEFT JOIN (SELECT z.product_id,
        TRUNC((SUM(CASE WHEN z.shipment_status = 'Early Shipment' THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 1) AS pct_early_shipments,
        TRUNC((SUM(CASE WHEN z.shipment_status = 'Late Shipment' THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 1) AS pct_late_shipments 
    FROM (
        SELECT i.product_id,
            CASE 
                WHEN DATE_PART('day', AGE(x.shipment_date, i.order_date)) >= 6 AND x.delivery_date IS NULL THEN 'Late Shipment'
                WHEN DATE_PART('day', AGE(x.shipment_date, i.order_date)) < 6 AND x.delivery_date IS NULL THEN 'Early Shipment'
                ELSE 'On Time'
            END AS shipment_status,
            i.order_date
        FROM
            niniroti9849_staging.shipments_deliveries x
        LEFT JOIN
            niniroti9849_staging.orders i ON x.order_id = i.order_id
    ) z
    GROUP BY z.product_id
) n ON CAST(n.product_id AS INTEGER) = CAST(a.product_id AS INTEGER)
LEFT JOIN (
    SELECT
        product_id,
        TRUNC((SUM(CASE WHEN review = 1 THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 1) AS pct_one_star_review,
        TRUNC((SUM(CASE WHEN review = 2 THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 1) AS pct_two_star_review,
        TRUNC((SUM(CASE WHEN review = 3 THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 1) AS pct_three_star_review,
        TRUNC((SUM(CASE WHEN review = 4 THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 1) AS pct_four_star_review,
        TRUNC((SUM(CASE WHEN review = 5 THEN 1 ELSE 0 END) * 100.0) / COUNT(*), 1) AS pct_five_star_review
    FROM
    niniroti9849_staging.reviews
    GROUP BY product_id
) g ON CAST(g.product_id AS INTEGER) = CAST(a.product_id AS INTEGER)
LEFT JOIN if_common.dim_products p ON CAST(p.product_id AS INTEGER) = CAST(a.product_id AS INTEGER)
Left join(select calendar_dt,case when day_of_the_week_num IN (1, 2, 3, 4, 5) AND working_day = 'false' 
		then TRUE ELSE FALSE END AS is_public_holiday
        from if_common.dim_dates)d on d.calendar_dt = a.order_date
LEFT JOIN (
    SELECT
        product_id,
        COUNT(review) AS no_of_reviews,
	    CAST(SUM(review) AS INTEGER) AS total_review_points
	    
    FROM
        niniroti9849_staging.reviews
    GROUP BY product_id
    ORDER BY no_of_reviews DESC
    LIMIT 1
) b ON CAST(a.product_id AS INTEGER) = CAST(b.product_id AS INTEGER)
WHERE
    a.product_id = '23' 
    AND a.quantity = '10';
"""

In [95]:
#execute query
cur.execute(sql_query)

In [96]:
# Fetch all rows from the query result
rows = cur.fetchall()

In [97]:
# Get the column names from the cursor description
column_names = [desc[0] for desc in cur.description]

In [102]:
# Define local file path to save CSV file
csv_file_path = '/Users/niniolarotimi/best_performing_product.csv'

# Write query result to CSV file, write both rows and columns
with open(csv_file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)   
    csv_writer.writerow(column_names)     
    csv_writer.writerows(rows)

In [72]:
# Create an S3 client
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

In [103]:
#Send csv file to s3 bucket. 
#Specify the folder path in the S3 bucket

s3_bucket = 'd2b-internal-assessment-bucket'
s3_folder = 'analytics_export/niniroti9849'  

# Combine the folder path and file name for the S3 key
s3_key = s3_folder + 'best_performing_product.csv'

# Upload CSV file to AWS S3
s3.upload_file(csv_file_path, s3_bucket, s3_key)

In [206]:
csv_file_path = '/Users/niniolarotimi/agg_shipments.csv'

In [207]:
# Create an S3 client
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

In [208]:
#Send csv file to s3 bucket. 
#Specify the folder path in the S3 bucket

s3_bucket = 'd2b-internal-assessment-bucket'
s3_folder = 'analytics_export/niniroti9849'  

# Combine the folder path and file name for the S3 key
s3_key = s3_folder + 'best_performing_product.csv'

# Upload CSV file to AWS S3
s3.upload_file(csv_file_path, s3_bucket, s3_key)

In [209]:
csv_file_path = '/Users/niniolarotimi/agg_public_holiday.csv'

In [210]:
#Send csv file to s3 bucket. 
#Specify the folder path in the S3 bucket

s3_bucket = 'd2b-internal-assessment-bucket'
s3_folder = 'analytics_export/niniroti9849'  

# Combine the folder path and file name for the S3 key
s3_key = s3_folder + 'agg_public_holiday.csv'

# Upload CSV file to AWS S3
s3.upload_file(csv_file_path, s3_bucket, s3_key)